# Tweet 1 - main tweet

🔎 Let's build a RAG-SQL agent that intelligently routes queries to a Text-to-SQL engine or uses retrieval over documents.

# Tweet 2

Here's what our RAG-SQL Agent can do:

- Answer questions from unstructured Wikipedia data using RAG
- Query structured city statistics using Text-to-SQL
- Automatically choose the right tool for each query

Let's see how it works! 🚀

{Demo GIF showing the assistant in action}

# Tweet 3

Here's the system architecture:

Tech stack:
- @Llama_Index for orchestration
- @Llama_Cloud for vector database and search
- @OpenAI for response generation
- @Streamlit for the UI

![System Architecture Diagram](assets/architecture.png)

# Tweet 4

0️⃣ Get our API keys:

- `platform.openai.com`
- `cloud.llamaindex.ai`

# Tweet 4

1️⃣ Set up our knowledge sources:

1️. A SQL database with city statistics (population, state)

2️. Wikipedia PDFs about major US cities indexed in LlamaCloud

Both will be queried through specialized tools selected by our router.

# Tweet 5

2️⃣ Let's set up our SQL database first. We'll create an in-memory SQLite database with information about major US cities:

```python
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, insert

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

city_stats_table = Table(
    "city_stats", metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("state", String(16), nullable=False),
)

# Add sample data below
...
```

# Tweet 6

3️⃣ Now we create a Text-to-SQL query engine using LlamaIndex that can translate natural language into SQL queries:

```python
from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine

sql_database = SQLDatabase(engine, include_tables=["city_stats"])
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"]
)
```

# Tweet 7

4️⃣ For the RAG component, we use LlamaCloud to index and query uploaded Wikipedia articles about major US cities, which we uploaded under the hood to an index:

```python
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

index = LlamaCloudIndex(
    name=os.getenv("LLAMA_CLOUD_INDEX"),
    project_name=os.getenv("LLAMA_CLOUD_PROJECT"),
    organization_id=os.getenv("LLAMA_CLOUD_ORG_ID"),
    api_key=os.getenv("LLAMA_CLOUD_API_KEY")
)

llama_cloud_query_engine = index.as_query_engine()
```

# Tweet 8

5️⃣ Now for the magic part - we create QueryEngineTools for both our engines which will be exposed to a router agent later:

```python
from llama_index.core.tools import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description="Useful for translating natural language into SQL queries over the city_stats table (population/state of US cities)",
    name="sql_tool"
)

llama_cloud_tool = QueryEngineTool.from_defaults(
    query_engine=llama_cloud_query_engine,
    description="Useful for answering semantic questions about certain cities in the US",
    name="llama_cloud_tool"
)
```

# Tweet 9

6️⃣ The heart of our system is the router agent that decides which tool to use based on question type. Let's look at how it is implemented:

```python
from llama_index.core.workflow import Workflow

class RouterOutputAgentWorkflow(Workflow):

    def __init__(self,
        tools,
        timeout,
        llm,
        chat_history,
    ):

        self.tools = tools
        self.tools_dict = {tool.metadata.name: tool for tool in self.tools}
        self.llm = llm or OpenAI(temperature=0, model="gpt-3.5-turbo")
        self.chat_history = chat_history or []
```

# Tweet 10

7️⃣ A critical moment happens when the router's `chat` method is invoked, leading to an OpenAI LLM choosing which tool to use:

```python
    ... # the rest of the class
    @step()
    async def chat(self, ev: InputEvent):

        # Put msg into LLM with tools included
        chat_res = await self.llm.achat_with_tools(
            self.tools,
            chat_history=self.chat_history,
            allow_parallel_tool_calls=True
        )

        # Tool calls
        tool_calls = self.llm.get_tool_calls_from_response(chat_res, error_on_no_tool_call=False)
        
        ai_message = chat_res.message
        self.chat_history.append(ai_message)

        # No tool calls, return chat message.
        if not tool_calls:
            return StopEvent(result=ai_message.content)

        return GatherToolsEvent(tool_calls=tool_calls)
```

# Tweet 11

8️⃣ Finally, we initialize an instance of the router, provide it with the tools we created and a user query:

```python
workflow = RouterOutputAgentWorkflow(
    tools=[sql_tool, llama_cloud_tool], verbose=True, timeout=120
)

result = await workflow.run(
    message="Which city has the highest population?"
)

print(result)
```

Output:

New York City has the highest population.

# Tweet 12

That's a wrap! Find the full code for this project from our GitHub repository👇

{LINK TO THE PROJECT CODE}

# Tweet 13

If you're interested in:

- Python 🐍
- Machine Learning 🤖
- AI Engineering ⚙️

Find me → @akshay_pachaar

Cheers! 🥂